In [1]:
!pip install pandas-datareader

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import time
import random
from tqdm import tqdm
from pandas_datareader import data
from scipy.stats import zscore
import matplotlib.pyplot as plt

In [3]:
!pip install yahoo-fin

In [ ]:
import yahoo_fin.stock_info as yf

In [ ]:
balance_sheet = []
income_statement = []
cfs = []
years = []

In [ ]:
def get_data(ticker):
    global balance_sheet
    global income_statement
    global cfs
    global years
    balance_sheet=yf.get_balance_sheet(ticker)
    income_statement=yf.get_income_statement(ticker)
    cfs=yf.get_cash_flow(ticker)
    years=balance_sheet.columns

## Testing for individual stock

In [ ]:
get_data('C09.SI')

In [ ]:
income_statement

# Piotroski-F-Score

In [ ]:
balance_sheet = []
income_statement = []
cfs = []
years = []
profitability_score=0
leverage_score=0
operating_efficiency_score=0
pe_ratio=0

# Fetch fin statements

In [10]:
def get_data(ticker):
    global balance_sheet
    global income_statement
    global cfs
    global years
    balance_sheet=yf.get_balance_sheet(ticker)
    income_statement=yf.get_income_statement(ticker)
    cfs=yf.get_cash_flow(ticker)
    years=balance_sheet.columns

# PE Ratios

In [11]:
def pe(ticker):
    global pe_ratio
    pe_ratio=yf.get_quote_table(ticker)['PE Ratio (TTM)']
    if pe_ratio != pe_ratio: #check if NaN
        pe_ratio=0

## Defining Profitability

In [12]:
def profitability():
    global profitability_score
    #Criteria 1
    # Scores 1 and 2 - Net income
    net_income=income_statement[years[0]]['netIncome']
    net_income_py=income_statement[years[1]]['netIncome']
    ni_score=1 if net_income > 0 else 0  
    ni_score_2=1 if net_income > net_income_py else 0
    
    #Criteria 2
    # Score 3 -operating cash flow
    op_cf = cfs[years[0]]['totalCashFromOperatingActivities']
    op_cf_score=1 if op_cf>0 else 0
    
    #Criteria 3
    #Score 4-Change in RoA
    avg_assets=(balance_sheet[years[0]]['totalAssets']+balance_sheet[years[1]]['totalAssets'])/2
    avg_assets_py=(balance_sheet[years[1]]['totalAssets']+balance_sheet[years[2]]['totalAssets'])/2
    RoA=net_income/avg_assets
    RoA_py=net_income_py/avg_assets_py
    RoA_score=1 if RoA > RoA_py else 0
    
    #Criteria 4
    #Score 5-Accruals
    total_assets=balance_sheet[years[0]]['totalAssets']
    accurals=op_cf/total_assets - RoA
    ac_score = 1 if accurals>0 else 0
    
    profitability_score=ni_score+ni_score_2+op_cf_score+RoA_score+ac_score
    print("Profitability Score: "+str(profitability_score))

## Defining Leverage

In [13]:
def leverage():
    global leverage_score
    #Criteria 5 (Modified long-term levarge ratio)
    #Score 6 - Long-term debt ratio
    try:
        lt_debt=balance_sheet[years[0]]['longTermDebt']
        total_assets=balance_sheet[years[0]]['totalAssets']
        debt_ratio=lt_debt/total_assets        
        debt_ratio_score=1 if debt_ratio<0.4 else 0        
    except:
        debt_ratio_score=1
    
        
    #Criteria 6 (Modified current ratio)
    #Score 7 -Current ratio
    current_assets=balance_sheet[years[0]]['totalCurrentAssets']
    current_liab=balance_sheet[years[0]]['totalCurrentLiabilities']
    current_ratio=current_assets/current_liab   
    current_ratio_score=1 if current_ratio>1 else 0
    
    leverage_score = debt_ratio_score + current_ratio_score
    print("Leverage Score: "+str(leverage_score))

## Defining Operating Efficiency

In [14]:
def operating_efficiency():
    global operating_efficiency_score
    #Criteria 8
    #Score 8-Gross margin
    gp=income_statement[years[0]]['grossProfit']
    gp_py=income_statement[years[1]]['grossProfit']
    revenue=income_statement[years[0]]['totalRevenue']
    revenue_py=income_statement[years[1]]['totalRevenue']
    gm=gp/revenue
    gm_py=gp_py/revenue_py
    gm_score=1 if gm>gm_py else 0
    
    #Criteria 9
    #Score 9-Asset Turnover
    avg_assets=(balance_sheet[years[0]]['totalAssets']+balance_sheet[years[1]]['totalAssets'])/2
    avg_assets_py=(balance_sheet[years[1]]['totalAssets']+balance_sheet[years[2]]['totalAssets'])/2
    asset_turnover=revenue/avg_assets  
    asset_turnover_py=revenue_py/avg_assets_py
    asset_turnover_score=1 if asset_turnover>asset_turnover_py else 0
    
    operating_efficiency_score=gm_score+asset_turnover_score
    print("Operating Efficiency: "+str(operating_efficiency_score))

In [ ]:
tickers=['CLN.SI','A30.SI','B61.SI','BTE.SI','OU8.SI','C29.SI','C33.SI','C09.SI','ADN.SI','TQ5.SI','586.SI','5PC.SI','F17.SI','5DP.SI'
        ,'5JK.SI','H13.SI','H30.SI','H78.SI','H19.SI','41O.SI','LJ3.SI','5UX.SI','OYY.SI','E8Z.SI','A26.SI','5IC.SI','U06.SI','5H0.SI'
        ,'AWI.SI','T24.SI','CHJ.SI','EH5.SI','U14.SI','W05.SI','Z25.SI','Z59.SI']
summary=pd.DataFrame(columns = ['Ticker', 'PE ratio', 'Profitability', 'Leverage', 'Operating efficiency'])
for ticker in tickers:
    try:
        get_data(ticker)
        pe(ticker)
        print(ticker)
        profitability()
        leverage()
        operating_efficiency()
        new_row={'Ticker':ticker,
                 'PE ratio':pe_ratio,
                 'Profitability':profitability_score,
                 'Leverage':leverage_score,
                 'Operating efficiency':operating_efficiency_score}
        summary=summary.append(new_row, ignore_index=True)
        print(ticker+' added.')    
        time.sleep(3)
    except:
        print(ticker+':Something went wrong.')

summary['Total score'] =summary['Profitability']+summary['Leverage']+summary['Operating efficiency']
summary.to_csv('Summary.csv')   

CLN.SI
Profitability Score: 5
Leverage Score: 2
Operating Efficiency: 0
CLN.SI added.
A30.SI
Profitability Score: 5
Leverage Score: 2
Operating Efficiency: 0
A30.SI added.
B61.SI
Profitability Score: 5
Leverage Score: 2
Operating Efficiency: 1
B61.SI added.
BTE.SI
Profitability Score: 3
Leverage Score: 2
Operating Efficiency: 0
BTE.SI added.


In [ ]:
summary

In [ ]:
test_summary=pd.DataFrame(columns = ['Ticker', 'PE ratio', 'Profitability', 'Leverage', 'Operating efficiency'])
test_tickers=['msft','aapl','googl','nio']
for ticker in test_tickers:
    try:
        get_data(ticker)
        pe(ticker)
        print(ticker)
        profitability()
        leverage()
        operating_efficiency()
        new_row={'Ticker':ticker,
                 'PE ratio':pe_ratio,
                 'Profitability':profitability_score,
                 'Leverage':leverage_score,
                 'Operating efficiency':operating_efficiency_score}
        test_summary=test_summary.append(new_row, ignore_index=True)
        print(ticker+' added.')    
#         time.sleep(3)
    except:
        print(ticker+':Something went wrong.')

test_summary['Total score'] =test_summary['Profitability']+test_summary['Leverage']+test_summary['Operating efficiency']

In [ ]:
test_summary